# Transhipment: maximum flow

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

import sys

import numpy as np
from matplotlib import pyplot as plt
from networkx import (
    DiGraph,
    draw_networkx_nodes,
    draw_networkx_edges,
    draw_networkx_edge_labels,
    Graph,
    draw_networkx_labels,
)
from teaching_optimization.simplex_tableau import SimplexAlgorithmTableau
from teaching_optimization.tableau import SimplexTableau


For security
reasons, the city council of Lausanne wants to know how many persons can get
from the train station to the bar ``Great Escape'' during one hour.  The possible links which can be used and their
corresponding capacity are the following:

- Station to Metro: 1500 persons/hour.
- Station to Place de l'Europe: 2700 persons/hour.
- Metro to the Great Escape: 2500 persons/hour.
- Place de l'Europe to Metro: 1800 persons/hour.
- Place de l'Europe to Great Escape: 2200 persons/hour.

We model this as a maximum flow problem.

# Question 1
Code and draw the corresponding network. The network must have the cost, the upper
and lower bounds on each arc, including the additional dummy arc used for counting the flow.

In [ ]:

the_network = DiGraph()


Add nodes

In [ ]:
the_network.add_node('Station')
the_network.add_node('Metro M2')
the_network.add_node('Pl. Europe')
the_network.add_node('Great Escape')


Add arcs with cost, and capacity (lower and upper bound). Here is how to code the first arc.

In [ ]:
the_network.add_edge('Station', 'Metro M2', cost=0, lower_bound=0, upper_bound=1500)

Add the other arcs

In [ ]:

the_network.add_edge(
    'Station', 'Pl. Europe', cost=0, lower_bound=0, upper_bound=2700
)
the_network.add_edge(
    'Pl. Europe', 'Metro M2', cost=0, lower_bound=0, upper_bound=1800
)
the_network.add_edge(
    'Pl. Europe', 'Great Escape', cost=0, lower_bound=0, upper_bound=2200
)
the_network.add_edge(
    'Metro M2', 'Great Escape', cost=0, lower_bound=0, upper_bound=2500
)


We save the list of arcs from the original network, to draw them differently.

In [ ]:
original_arcs = list(the_network.edges())


Add the dummy arc.
Cost = -1, and infinite capacity.

In [ ]:

the_network.add_edge(
    'Great Escape', 'Station', cost=-1, lower_bound=0, upper_bound=np.inf, dummy=1
)



Define coordinates for the plot.

In [ ]:
pos = {
    'Station': (0, 4),
    'Metro M2': (5, 4),
    'Pl. Europe': (3, 0),
    'Great Escape': (8, 0),
}



Function to plot the network.

In [ ]:
def plot_network(network: Graph) -> None:
    """Plot the network and its data.

    :param network: network to plot.
    """

    # Figure size
    plt.figure(figsize=(10, 10))

    # Draw the nodes
    draw_networkx_nodes(network, pos, node_size=1000, node_color='lightblue', alpha=0.5)

    # Move the labels
    shifted_positions = {
        node: (coord[0], coord[1] - 0.2) for node, coord in pos.items()
    }

    # Draw the node labels
    draw_networkx_labels(network, shifted_positions, font_size=12, font_weight='bold')

    # Draw the arcs with labels
    edge_labels = {}
    for u, v, data in network.edges(data=True):
        lower_bound = data['lower_bound']
        upper_bound = data['upper_bound']
        label = f'({lower_bound},{upper_bound})'
        edge_labels[(u, v)] = label

    draw_networkx_edges(
        network, pos, edgelist=original_arcs, arrowstyle='->', arrowsize=20
    )
    # We draw the dummy arc differently.
    dummy_arc = ('Great Escape', 'Station')
    draw_networkx_edges(
        network,
        pos,
        edgelist=[dummy_arc],
        style='dotted',
        arrowstyle='->',
        arrowsize=20,
    )
    draw_networkx_edge_labels(
        network, pos, edge_labels=edge_labels, font_size=10, label_pos=0.3
    )

    # Display the graph
    plt.title('Maximum flow problem')
    plt.axis('off')
    plt.show()


plot_network(network=the_network)



# Question 2
Write down the corresponding optimization problem. Remember that
the max flow is a special case of the transhipment problem.

A transhipment problem is written:
$$\min_{x} \sum_{(i,j)\in \mathcal{A}} c_{ij}x_{ij}$$
subject to the conservation constraints,
$$div(x)_i = s_i  \quad \forall i \in \mathcal{N},$$
and the bounds on the flows:
$$\ell_{ij} \leq x_{ij} \leq u_{ij},\;\forall (i,j) \in \mathcal{A},$$
where $\mathcal{A}$ is the set of arcs in the network, $\mathcal{N}$
the set of nodes, $\ell_{ij}$ and $u_{ij}$ the lower and upper bounds on
the flows from node $i$ to node $j$. For the maximum flow problem,
since the costs are 0 on all arcs except the arc returning the flow
from the destination to the origin, the objective function is reduced
to:
$$\min_{x} -x_{GE, S}.$$

The bound constraints on the arcs induce the following inequality constraints:
\begin{align*}
x_{S, M2} &\leq 1500, \\
x_{S, E} &\leq 2700, \\
x_{E, M2} &\leq 1800, \\
x_{E, GE} &\leq 2200, \\
x_{M2, GE} &\leq 2500, \\
x_i &\geq 0, \quad\forall i.
\end{align*}

The flow conservation at each node induces the following equality constraints:
\begin{align*}
-x_{GE, S} + x_{S, M2} + x_{S, E} &= 0, \qquad \text{(Station)}\\
-x_{S, E} + x_{E, M2} + x_{E, GE} & = 0, \qquad \text{(Pl. Europe)}\\
-x_{S, M2} - x_{E, M2} + x_{M2, GE} & = 0, \qquad \text{(Metro M2)}\\
-x_{M2, GE} - x_{E, GE} + x_{GE, S} & = 0. \qquad \text{(Great Escape)}
\end{align*}

The optimization problem is therefore:
$$
\min_{x\in\mathbb{R}^5} -x_{GE, S}
$$
subject to
\begin{align*}
-x_{GE, S} + x_{S, M2} + x_{S, E} &= 0,\\
-x_{S, E} + x_{E, M2} + x_{E, GE} & = 0,\\
-x_{S, M2} - x_{E, M2} + x_{M2, GE} & = 0,\\
-x_{M2, GE} - x_{E, GE} + x_{GE, S} & = 0,\\
x_{S, M2} &\leq 1500, \\
x_{S, E} &\leq 2700, \\
x_{E, M2} &\leq 1800, \\
x_{E, GE} &\leq 2200, \\
x_{M2, GE} &\leq 2500, \\
x_{S,M2}, x_{S,E}, x_{E,M2}, x_{E,GE}, x_{M2,GE}  &\geq 0.
\end{align*}

# Question 3
Solve the problem with the simplex algorithm
A solution to this problem is the following flows:

- $x_{S, M2} = 1500$,
- $x_{S, E} = 2700$,
- $x_{E, M2} = 500$,
- $x_{E, GE} = 2200$,
- $x_{M2, GE} = 2000$.

Another solution is

- $x_{S, M2} = 1500$,
- $x_{S, E} = 2700$,
- $x_{E, M2} = 1000$,
- $x_{E, GE} = 1700$,
- $x_{M2, GE} = 2500$.

Check that they both have the same value for the objective function.

As the problem is solved by the simplex algorithm, no need to transform the network to obtain the standard form.
Simply add slack variables like for general linear optimization problems, even if the matrix of the standard form
problem is not the incidence matrix of a transhipment problem.

The optimization problem in standard form is:
$$
\min_{x\in\mathbb{R}^5} -x_{GE, S}
$$
subject to
\begin{align*}
-x_{GE, S} + x_{S, M2} + x_{S, E} &= 0,\\
-x_{S, E} + x_{E, M2} + x_{E, GE} & = 0,\\
-x_{S, M2} - x_{E, M2} + x_{M2, GE} & = 0,\\
-x_{M2, GE} - x_{E, GE} + x_{GE, S} & = 0,\\
x_{S, M2} + e_{S, M2}&= 1500, \\
x_{S, E} + e_{S, E} &= 2700, \\
x_{E, M2}+ e_{E, M2} &= 1800, \\
x_{E, GE} + e_{E, GE} &= 2200, \\
x_{M2, GE} + e_{M2, GE} &= 2500, \\
x_{S,M2}, x_{S,E}, x_{E,M2}, x_{E,GE}, x_{M2,GE}, x_{GE, S}  &\geq 0, \\
e_{S, M2}, e_{S, E}, e_{E, M2}, e_{E, GE}, e_{M2, GE} & \geq 0.
\end{align*}

The matrix is

| $x_{GE, S}$ |$x_{S,M2}$ | $x_{S,E}$ | $x_{E,M2}$ | $x_{E,GE}$ | $x_{M2,GE}$ | $e_{S, M2}$ | $e_{S, E}$ | $e_{E, M2}$ | $e_{E, GE}$ | $e_{M2, GE}$ |
|:-----------:|:---------:|:---------:|:----------:|:----------:|:-----------:|:-----------:|:----------:|:-----------:|:-----------:|:------------:|
|     -1      |     1     |     1     |       0    |     0      |      0      |      0      |      0     |      0      |      0      |      0       |
|      0      |     0     |     -1    |       1    |     1      |      0      |      0      |      0     |      0      |      0      |      0       |
|      0      |    -1     |     0     |      -1    |     0      |      1      |      0      |      0     |      0      |      0      |      0       |
|      1      |     0     |     0     |       0    |    -1      |     -1      |      0      |      0     |      0      |      0      |      0       |
|      0      |     1     |     0     |       0    |     0      |      0      |      1      |      0     |      0      |      0      |      0       |
|      0      |     0     |     1     |       0    |     0      |      0      |      0      |      1     |      0      |      0      |      0       |
|      0      |     0     |     0     |       1    |     0      |      0      |      0      |      0     |      1      |      0      |      0       |
|      0      |     0     |     0     |       0    |     1      |      0      |      0      |      0     |      0      |      1      |      0       |
|      0      |     0     |     0     |       0    |     0      |      1      |      0      |      0     |      0      |      0      |      1       |

In [ ]:


matrix = np.array(
    [
        [-1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, -1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, -1, 0, -1, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
    ]
)
print(matrix)



The cost vector $c$.
It combines the actual cost for the flow (non zero only for the dummy arc, actually), and zeros for the slack
variables.

In [ ]:
vector_c = np.array([-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])



The right-hand-side $b$.
First: a zero for each node, in order to impose a circulation.
Second: the upper bound on each original arc.

In [ ]:
vector_b = np.array([0, 0, 0, 0, 1500, 2700, 1800, 2200, 2500])


We create the algorithm

In [ ]:
the_algorithm = SimplexAlgorithmTableau(
    objective=vector_c,
    constraint_matrix=matrix,
    right_hand_side=vector_b,
)


We solve the problem

In [ ]:
optimal_tableau: SimplexTableau = the_algorithm.solve()


Check if the problem is feasible

In [ ]:
if optimal_tableau is None:
    print(f'Optimization problem is infeasible.')
    sys.exit()


Optimal solution

In [ ]:
print(optimal_tableau.feasible_basic_solution)


Interpretation in terms of arc flows:

In [ ]:
flow_ge_s = optimal_tableau.feasible_basic_solution[0]
print(f'GE -> S: {flow_ge_s}')

flow_s_m2 = optimal_tableau.feasible_basic_solution[1]
print(f'S -> M2: {flow_s_m2}')

flow_s_e = optimal_tableau.feasible_basic_solution[2]
print(f'S -> E : {flow_s_e}')

flow_e_m2 = optimal_tableau.feasible_basic_solution[3]
print(f'E ->M2 : {flow_e_m2}')

flow_e_ge = optimal_tableau.feasible_basic_solution[4]
print(f'E ->GE : {flow_e_ge}')

flow_m2_ge = optimal_tableau.feasible_basic_solution[5]
print(f'M2 ->GE: {flow_m2_ge}')



Optimal value

In [ ]:
print(f'{optimal_tableau.value_objective_function}')


What is the total flow through the network?
We need to change the sign, as we solved a minimization problem.

In [ ]:
total_flow = -optimal_tableau.value_objective_function
print(f'Total flow through the network: {total_flow}')



# Question 4
Identify a saturated cut in the original graph, that does not involve
the dummy arc.

A saturated cut is a cut where all arcs are saturated, hence a
saturated cut is a cut where the flow is equal to the capacity of
the cut. The cut $\Gamma = ( \mathcal{M} , \mathcal{N}
\setminus \mathcal{M})$, where $\mathcal{M}= \{\text{Station} \}$ is
a saturated cut. Indeed, the two arcs connecting a node from
$\mathcal{M}$ with a node from  $\mathcal{N}
\setminus \mathcal{M}$ are saturated.